In [1]:
import numpy as np
import freud
import networkx as nx
import matplotlib.pyplot as plt
import os
import time
from collections import defaultdict
from tqdm import tqdm
from matplotlib.patches import Polygon

# Конфигурационные параметры
OUTPUT_DIR = "C:/Users/ZL/Desktop/практика"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Установка случайного зерна для воспроизводимости
np.random.seed(42)

# Улучшенная функция нормализации координат (с обработкой периодических границ)
def normalization(coord):
    x, y = coord[0], coord[1]
    
    # Отображение на основной период [0, 1)
    x = x % 1.0
    y = y % 1.0
    
    # Обработка проблем с точностью чисел с плавающей запятой
    x = round(x, 8)
    y = round(y, 8)
    
    return (x, y)

# Генерация набора точек (с периодическими границами)
def generate_points(N):
    points = np.vstack((np.random.uniform(0, 1, N), np.random.uniform(0, 1, N))).T
    points = np.hstack((points, np.zeros((N, 1))))  # Добавление z=0
    box = freud.box.Box(Lx=1, Ly=1, is2D=True)
    points = box.wrap(points)  # Применение периодических границ
    return box, points

# Преобразование вершин ячеек
def transform(cells):
    ans = []
    for cell in cells:
        # Сохраняем только допустимые многоугольники (количество вершин >= 3)
        if len(cell) >= 3:
            ans.append([normalization(x) for x in cell])
    return ans

# Получение уникальных вершин и их отображения
def get_vertices(cells):
    vertices = []
    for cell in cells:
        # Фильтрация недопустимых ячеек
        if len(cell) >= 3:
            vertices.extend(cell)
    
    # Удаление дубликатов с использованием множества
    unique_vertices = set(vertices)
    
    # Создание отображения
    return {vertex: idx for idx, vertex in enumerate(unique_vertices)}

# Извлечение информации о рёбрах
def get_edges(vertices, transformed_cells):
    edges = set()  # Автоматическое удаление дубликатов с использованием множества
    
    for cell in transformed_cells:
        num_vertices = len(cell)
        # Замыкание многоугольника (добавление первой вершины в конец)
        closed_cell = cell + [cell[0]]
        
        for i in range(num_vertices):
            v1 = closed_cell[i]
            v2 = closed_cell[i+1]
            
            # Убедимся, что вершины есть в словаре
            if v1 in vertices and v2 in vertices:
                # Создание нормализованного представления ребра (сортировка индексов вершин)
                idx1 = vertices[v1]
                idx2 = vertices[v2]
                edge = tuple(sorted((idx1, idx2)))
                edges.add(edge)
    
    return list(edges)

# Создание объекта графа
def create_graph(vertices, edges):
    G = nx.Graph()
    G.add_nodes_from(vertices.values())
    G.add_edges_from(edges)
    return G

# Вычисление максимальной степени графа
def get_max_degree(G):
    """Вычисление максимальной степени графа"""
    return max(dict(G.degree).values())

# Улучшенный алгоритм раскраски рёбер
def optimal_edge_coloring(G):
    """
    Раскраска рёбер графа с использованием улучшенного алгоритма
    Возвращает: 
        edge_colors - словарь сопоставления рёбер с цветами
        num_colors - количество использованных цветов
    """
    # Вычисление максимальной степени и нижней границы хроматического индекса
    max_deg = max(dict(G.degree).values())
    
    # Попытка раскраски с использованием χ(G) цветов
    for colors in range(max_deg, max_deg + 2):
        edge_colors = {}
        vertex_color_usage = defaultdict(set)
        
        # Попытка раскрасить каждое ребро
        success = True
        for u, v in G.edges():
            # Поиск доступных цветов
            available_colors = set(range(colors))
            available_colors -= vertex_color_usage[u]
            available_colors -= vertex_color_usage[v]
            
            if not available_colors:
                success = False
                break
            
            # Выбор минимального доступного цвета
            color = min(available_colors)
            edge_colors[(u, v)] = color
            vertex_color_usage[u].add(color)
            vertex_color_usage[v].add(color)
        
        if success:
            return edge_colors, colors
    
    # В случае неудачи используем жадный алгоритм с max_deg+1 цветами
    return greedy_edge_coloring(G)

# Жадный алгоритм раскраски рёбер (резервный)
def greedy_edge_coloring(G):
    # Инициализация структур данных
    edge_colors = {}
    vertex_color_usage = defaultdict(set)
    
    # Обработка вершин в порядке убывания степени
    sorted_edges = sorted(G.edges(), key=lambda e: max(G.degree[e[0]], G.degree[e[1]]), reverse=True)
    
    # Обход всех рёбер
    for u, v in sorted_edges:
        # Поиск минимального цвета, не использованного на u и v
        available_colors = set()
        max_possible_color = max(len(vertex_color_usage[u]), len(vertex_color_usage[v])) + 1
        
        # Попробуем цвета от 0 до максимально возможного
        for c in range(max_possible_color + 1):
            if c not in vertex_color_usage[u] and c not in vertex_color_usage[v]:
                available_colors.add(c)
        
        # Если доступных цветов нет, создаем новый
        if not available_colors:
            color = max_possible_color
        else:
            color = min(available_colors)
        
        # Назначение цвета
        edge_colors[(u, v)] = color
        vertex_color_usage[u].add(color)
        vertex_color_usage[v].add(color)
    
    # Вычисление фактического количества использованных цветов
    num_colors = len(set(edge_colors.values()))
    
    return edge_colors, num_colors

# Генерация диаграммы Вороного и вычисление хроматического индекса
def generate_voronoi_and_compute_ci(N):
    """Генерация диаграммы Вороного и вычисление её хроматического индекса"""
    # Генерация точек и вычисление диаграммы Вороного
    box, points = generate_points(N)
    voro = freud.locality.Voronoi()
    voro.compute((box, points))
    cells = voro.polytopes
    
    # Преобразование ячеек и получение вершин и рёбер
    transformed_cells = transform(cells)
    
    # Проверка наличия допустимых ячеек
    if not transformed_cells:
        return None
    
    vertices_dict = get_vertices(transformed_cells)
    edges = get_edges(vertices_dict, transformed_cells)
    
    # Создание объекта графа
    G = create_graph(vertices_dict, edges)
    
    # Вычисление свойств графа
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    max_degree = get_max_degree(G)
    
    # Раскраска рёбер (вычисление хроматического индекса)
    try:
        _, chromatic_index = optimal_edge_coloring(G)
    except Exception as e:
        print(f"Ошибка раскраски: {e}")
        return None
    
    # Сохранение всей релевантной информации
    voronoi_data = {
        "box": box,
        "points": points,
        "voro": voro,
        "cells": cells,
        "transformed_cells": transformed_cells,
        "vertices_dict": vertices_dict,
        "edges": edges,
        "graph": G,
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "max_degree": max_degree,
        "chromatic_index": chromatic_index
    }
    
    return voronoi_data

# Визуализация диаграммы Вороного и сохранение
def plot_voronoi_diagram(voronoi_data, save_path, title="Диаграмма Вороного"):
    """Визуализация диаграммы Вороного и её набора точек"""
    box = voronoi_data["box"]
    points = voronoi_data["points"]
    cells = voronoi_data["cells"]
    
    plt.figure(figsize=(8, 8))
    
    # Визуализация набора точек
    plt.scatter(points[:, 0], points[:, 1], s=50, c='red', zorder=10)
    
    # Визуализация многоугольников Вороного
    for cell in cells:
        polygon = Polygon(cell[:, :2], closed=True, 
                         fill=True, alpha=0.3, edgecolor='blue')
        plt.gca().add_patch(polygon)
    
    # Установка границ
    plt.xlim(-0.1, 1.1)
    plt.ylim(-0.1, 1.1)
    plt.title(title)
    plt.gca().set_aspect('equal')
    plt.grid(True, alpha=0.3)
    
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Визуализация структуры графа и сохранение
def plot_graph_structure(voronoi_data, save_path, title="Структура графа"):
    """Визуализация структуры графа"""
    G = voronoi_data["graph"]
    
    plt.figure(figsize=(8, 8))
    
    # Использование spring layout
    pos = nx.spring_layout(G, seed=42)
    
    # Визуализация узлов и рёбер
    nx.draw_networkx_nodes(G, pos, node_size=300, node_color='lightblue')
    nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.7)
    nx.draw_networkx_labels(G, pos, font_size=10)
    
    plt.title(title)
    plt.axis('off')
    
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Визуализация результатов раскраски и сохранение
def plot_edge_coloring(voronoi_data, save_path, title="Раскраска рёбер"):
    """Визуализация результатов раскраски рёбер"""
    G = voronoi_data["graph"]
    
    # Вычисление раскраски рёбер
    edge_colors, num_colors = optimal_edge_coloring(G)
    
    plt.figure(figsize=(8, 8))
    
    # Получение позиций узлов
    pos = nx.spring_layout(G, seed=42)
    
    # Подготовка цветовой карты
    color_list = plt.cm.tab10(np.linspace(0, 1, num_colors))
    
    # Назначение цвета каждому ребру
    edge_color_list = []
    for u, v in G.edges():
        color_idx = edge_colors[(u, v)]
        edge_color_list.append(color_list[color_idx])
    
    # Визуализация графа
    nx.draw_networkx_nodes(G, pos, node_size=300, node_color='lightblue')
    nx.draw_networkx_edges(G, pos, width=2, edge_color=edge_color_list)
    nx.draw_networkx_labels(G, pos, font_size=10)
    
    plt.title(f"{title}\nХроматический индекс = {num_colors}")
    plt.axis('off')
    
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Классификация на основе канонической формы графа
def classify_by_isomorphism(voronoi_list):
    """
    Классификация диаграмм Вороного на основе изоморфизма графов
    
    Параметры:
        voronoi_list: список данных диаграмм Вороного
        
    Возвращает:
        dict: отображение классов эквивалентности {ID класса: [индексы графов в классе]}
        list: список представителей классов
    """
    # Хранение классов эквивалентности
    classes = defaultdict(list)
    class_reps = {}
    class_id = 0
    
    # Индикатор выполнения
    pbar = tqdm(total=len(voronoi_list), desc="Классификация по изоморфизму")
    
    # Хранение канонических форм графов
    canonical_forms = {}
    
    # Обход всех графов
    for idx, voronoi_data in enumerate(voronoi_list):
        G = voronoi_data["graph"]
        
        # Генерация канонической формы графа
        canonical_form = nx.weisfeiler_lehman_graph_hash(G)
        
        # Проверка существования класса с такой канонической формой
        if canonical_form in canonical_forms:
            class_id_existing = canonical_forms[canonical_form]
            classes[class_id_existing].append(idx)
        else:
            # Создание нового класса
            classes[class_id].append(idx)
            class_reps[class_id] = voronoi_data
            canonical_forms[canonical_form] = class_id
            class_id += 1
        
        pbar.update(1)
    
    pbar.close()
    return dict(classes), class_reps

# Сбор графов Вороного с хроматическим индексом 3
def collect_chromatic_index_3_graphs(num_samples, N):
    """
    Сбор графов Вороного с хроматическим индексом 3
    
    Параметры:
        num_samples: количество генерируемых образцов
        N: размер набора точек
        
    Возвращает:
        list: список данных диаграмм Вороного с хроматическим индексом 3
    """
    chromatic_3_graphs = []
    
    # Индикатор выполнения
    pbar = tqdm(total=num_samples, desc="Сбор графов с χ'=3")
    
    for _ in range(num_samples):
        voronoi_data = generate_voronoi_and_compute_ci(N)
        
        if voronoi_data is not None and voronoi_data["chromatic_index"] == 3:
            chromatic_3_graphs.append(voronoi_data)
        
        pbar.update(1)
    
    pbar.close()
    
    print(f"Найдено {len(chromatic_3_graphs)} графов с хроматическим индексом 3")
    return chromatic_3_graphs


# Сохранение визуализации представителей классов эквивалентности
def save_equivalence_class_visualization(class_id, rep, output_dir):
    """Сохранение трёх видов визуализации для представителя класса эквивалентности"""
    # Создание директории класса
    class_dir = os.path.join(output_dir, f"class_{class_id}")
    os.makedirs(class_dir, exist_ok=True)
    
    # Диаграмма Вороного
    voronoi_path = os.path.join(class_dir, f"class_{class_id}_voronoi.png")
    plot_voronoi_diagram(
        rep, 
        voronoi_path,
        title=f"Класс эквивалентности {class_id}\nДиаграмма Вороного (N={len(rep['points'])})"
    )
    
    # Структура графа
    graph_path = os.path.join(class_dir, f"class_{class_id}_graph.png")
    plot_graph_structure(
        rep, 
        graph_path,
        title=f"Класс эквивалентности {class_id}\nСтруктура графа\nУзлы={rep['num_nodes']}, Рёбра={rep['num_edges']}"
    )
    
    # Раскраска рёбер
    coloring_path = os.path.join(class_dir, f"class_{class_id}_coloring.png")
    plot_edge_coloring(
        rep, 
        coloring_path,
        title=f"Класс эквивалентности {class_id}\nРаскраска рёбер\nΔ={rep['max_degree']}, χ'={rep['chromatic_index']}"
    )
    
    return {
        "class_id": class_id,
        "voronoi_path": voronoi_path,
        "graph_path": graph_path,
        "coloring_path": coloring_path
    }

# Основная программа
def main():
    # Ввод пользователя
    try:
        N = int(input("Введите размер набора точек N (по умолчанию 8): ") or 8)
        num_samples = int(input("Введите количество генерируемых образцов (по умолчанию 500): ") or 500)
    except ValueError:
        print("Недопустимый ввод, используются значения по умолчанию")
        N = 8
        num_samples = 500
    
    print("=" * 60)
    print(f"Анализ хроматического индекса диаграмм Вороного и исследование классов эквивалентности (N={N})")
    print("=" * 60)
    
    # Запись времени начала
    start_time = time.time()
    
    # Шаг 1: Сбор графов с хроматическим индексом 3
    chromatic_3_graphs = collect_chromatic_index_3_graphs(num_samples, N)
    
    if not chromatic_3_graphs:
        print("Графы с хроматическим индексом 3 не найдены, попробуйте увеличить количество образцов или изменить параметры")
        return
    
    # Шаг 2: Классификация на основе изоморфизма графов
    classes, class_reps = classify_by_isomorphism(chromatic_3_graphs)
    
    print(f"\nОбнаружено {len(classes)} классов эквивалентности")
    
    # Шаг 3: Сохранение визуализации классов эквивалентности
    class_info = []
    for class_id in classes:
        class_info.append(save_equivalence_class_visualization(
            class_id, class_reps[class_id], OUTPUT_DIR
        ))
    
    # Шаг 4: Генерация отчёта анализа
    report_path = os.path.join(OUTPUT_DIR, "analysis_report.txt")
    with open(report_path, "w", encoding='utf-8') as f:
        f.write("Отчёт анализа классов эквивалентности диаграмм Вороного\n")
        f.write("=" * 60 + "\n")
        f.write(f"Время анализа: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Размер набора точек (N): {N}\n")
        f.write(f"Количество сгенерированных образцов: {num_samples}\n")
        f.write(f"Количество графов с хроматическим индексом 3: {len(chromatic_3_graphs)}\n")
        f.write(f"Количество классов эквивалентности: {len(classes)}\n\n")
        
        f.write("Статистика классов эквивалентности:\n")
        f.write("ID\tРазмер\tУзлы\tРёбра\tМакс. степень\tХром. индекс\n")
        f.write("-" * 60 + "\n")
        
        for class_id, graph_indices in classes.items():
            rep = class_reps[class_id]
            class_size = len(graph_indices)
            
            f.write(f"{class_id}\t{class_size}\t{rep['num_nodes']}\t")
            f.write(f"{rep['num_edges']}\t{rep['max_degree']}\t{rep['chromatic_index']}\n")
        
        f.write("\nФайлы визуализации:\n")
        for info in class_info:
            f.write(f"\nКласс эквивалентности {info['class_id']}:\n")
            f.write(f"  Диаграмма Вороного: {info['voronoi_path']}\n")
            f.write(f"  Структура графа: {info['graph_path']}\n")
            f.write(f"  Раскраска рёбер: {info['coloring_path']}\n")
    
    # Вычисление времени анализа
    analysis_time = time.time() - start_time
    
    print("\n" + "=" * 60)
    print(f"Анализ завершён! Затраченное время: {analysis_time:.2f} секунд")
    print(f"Все результаты сохранены в директории: {OUTPUT_DIR}")
    print(f"Отчёт анализа: {report_path}")
    print("=" * 60)
    
    # Вывод сводки классов эквивалентности
    print("\nСводка классов эквивалентности:")
    print("ID\tРазмер\tУзлы\tРёбра\tМакс. степень\tХром. индекс")
    print("-" * 60)
    for class_id, graph_indices in classes.items():
        rep = class_reps[class_id]
        class_size = len(graph_indices)
        print(f"{class_id}\t{class_size}\t{rep['num_nodes']}\t{rep['num_edges']}\t{rep['max_degree']}\t{rep['chromatic_index']}")

if __name__ == "__main__":

    main()    

Введите размер набора точек N (по умолчанию 8):  3
Введите количество генерируемых образцов (по умолчанию 500):  


Анализ хроматического индекса диаграмм Вороного и исследование классов эквивалентности (N=3)


Сбор графов с χ'=3: 100%|███████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 453.38it/s]


Найдено 194 графов с хроматическим индексом 3


Классификация по изоморфизму:   0%|                                                            | 0/194 [00:00<?, ?it/s]C:\Users\ZL\AppData\Local\Programs\Python\Python312\Lib\site-packages\networkx\algorithms\graph_hashing.py:211: UserWarning: The hashes produced for graphs without node or edge attributeschanged in v3.5 due to a bugfix (see documentation).
  node_labels = _init_node_labels(G, edge_attr, node_attr)
Классификация по изоморфизму: 100%|████████████████████████████████████████████████| 194/194 [00:00<00:00, 8304.79it/s]



Обнаружено 3 классов эквивалентности

Анализ завершён! Затраченное время: 4.17 секунд
Все результаты сохранены в директории: C:/Users/ZL/Desktop/практика
Отчёт анализа: C:/Users/ZL/Desktop/практика\analysis_report.txt

Сводка классов эквивалентности:
ID	Размер	Узлы	Рёбра	Макс. степень	Хром. индекс
------------------------------------------------------------
0	163	6	9	3	3
1	24	6	8	3	3
2	7	6	6	2	3
